In [904]:
import random
import pandas as pd
from io import StringIO
import re
import yaml
import xgboost
import joblib
import numpy as np
import pandas as pd
players = pd.read_csv('C:/Github/nfl_sim/data/depth.csv',
                      usecols=['season', 'club_code', 'week',  'dense_depth','formation', 'gsis_id',
       'position', 'full_name'])
rb_stats =  pd.read_csv('data/rb_stats.csv',index_col=0)

In [905]:
rb_stats.sample(3)

,team_name,season,week,game_type,gsis_id_RB_1,gsis_id_RB_2,gsis_id_RB_0,full_name_RB_1,full_name_RB_2,full_name_RB_0,ppr_roll_RB_1,ppr_roll_RB_2,ppr_roll_RB_0,rush_roll_RB_1,rush_roll_RB_2,rush_roll_RB_0,carries_roll_RB_1,carries_roll_RB_2,carries_roll_RB_0
5620,NE,2021,9.0,REG,00-0036875,00-0029239,00-0035657,Rhamondre Stevenson,Brandon Bolden,Damien Harris,6.860000,8.666667,13.633333,27.200000,17.333333,61.833333,7.000000,3.000000,14.666667
5566,NE,2018,14.0,REG,00-0034845,00-0030288,00-0031062,Sony Michel,Rex Burkhead,James White,10.733333,4.950000,15.616666,68.666667,21.000000,25.500000,16.166667,6.333333,6.666667
6190,NYG,2022,7.0,REG,00-0033308,00-0036569,00-0034844,Matt Breida,Gary Brightwell,Saquon Barkley,2.300000,2.800000,18.033334,7.333333,9.000000,93.666667,2.666667,1.666667,20.833333


In [906]:
class QB:
    def __init__(self, d):  # noqa: F811
        self.name = d["full_name"]
        self.attempts = 0
        self.completions = 0
        self.rush_yards = 0
        self.pass_yards = 0
        self.td = 0
        self.int = 0

    def __repr__(self):
        return f"QB:{self.name} has {self.completions} completions for {self.pass_yards} yards"


class RB:
    def __init__(self, d):
        self.name = d["full_name"]
        self.id = d["gsis_id"]
        self.depth_team= int(d['dense_depth'])
        self.rush_yards = 0
        self.receiving_yards = 0
        self.targets = 0
        self.receptions = 0
        self.td = 0
        self.carries = 0

    def __repr__(self):
        return f"RB:{self.name} has {self.carries} carries for {self.rush_yards} yards"


class WR:
    def __init__(self, d):
        self.name = d["full_name"]
        self.id = d["gsis_id"]
        self.rush_yards = 0
        self.receiving_yards = 0
        self.targets = 0
        self.receptions = 0
        self.td = 0
        self.carries = 0

    def __repr__(self):
        return f"WR:{self.name} has {self.receptions} receptions for {self.receiving_yards} yards"


class Team:
    def __init__(self, name, season, week, use_current_injuries=False):
        self.name = name
        self.score = 0
        self.plays = 0
        self.players = players.loc[
            (players.club_code == name) & (players.season == season)
        ]
        self.players = self.players.loc[
            (self.players.week == min(self.players.week.max(), week))
            & (self.players.formation == "Offense")
            & (self.players.position.isin(["QB", "WR", "TE", "RB"]))
        ]

        self.QBs = self.get_players_by_position("QB")
        self.RBs = self.get_players_by_position("RB")
        self.WRs = self.get_players_by_position("WR")
        self.rb_stats = rb_stats.loc[
            (rb_stats.team_name == name) & (rb_stats.season == season)
        ]
        self.rb_stats = self.rb_stats.loc[
            (self.rb_stats.week == min(self.rb_stats.week.max(), week))
        ].to_dict(orient='records')[0]

    def get_players_by_position(self, position):
        """Filter players by position and create player objects."""
        position_data = self.players[(self.players["position"] == position)]

        # Create player objects based on position
        players = []
        for _, player_data in position_data.iterrows():
            if position == "WR":
                players.append(WR(player_data))
            elif position == "RB":
                players.append(RB(player_data))
            elif position == "QB":
                players.append(QB(player_data))
        return players

In [907]:
vik = Team('PHI',2024, 13)
det = Team('DET',2024, 13)
vik.players

,season,club_code,week,formation,gsis_id,position,full_name,dense_depth
531662,2024,PHI,12.0,Offense,00-0034351,TE,Dallas Goedert,0.0
531666,2024,PHI,12.0,Offense,00-0035676,WR,A.J. Brown,0.0
531670,2024,PHI,12.0,Offense,00-0036912,WR,DeVonta Smith,1.0
531672,2024,PHI,12.0,Offense,00-0037741,WR,Jahan Dotson,2.0
531684,2024,PHI,12.0,Offense,00-0034844,RB,Saquon Barkley,0.0
531685,2024,PHI,12.0,Offense,00-0036389,QB,Jalen Hurts,0.0
531692,2024,PHI,12.0,Offense,00-0037086,TE,Grant Calcaterra,1.0
531693,2024,PHI,12.0,Offense,00-0036919,RB,Kenneth Gainwell,1.0
531698,2024,PHI,12.0,Offense,00-0039236,WR,Johnny Wilson,3.0
531709,2024,PHI,12.0,Offense,00-0038102,QB,Kenny Pickett,1.0


In [908]:
vik.RBs[2].depth_team

2

In [909]:
det.RBs

[RB:David Montgomery has 0 carries for 0 yards,
 RB:Jahmyr Gibbs has 0 carries for 0 yards,
 RB:Craig Reynolds has 0 carries for 0 yards,
 RB:Sione Vaki has 0 carries for 0 yards]

In [910]:
class GameState:


	def __init__(self, home, away, config):
		self.home =home
		self.away = away
		self.quarter = 1
		self.possession = None
		self.down = 1
		self.togo = 10
		self.ball_position = 35  # Yardline (0-100, 50 is midfield)
		self.clock = 900  # Seconds in the current quarter (15 mins = 900 seconds)
		self.home_timeouts = 3
		self.away_timeouts = 3
		self.pbp = []
		self.run_or_pass = joblib.load('models/run_or_pass.joblib')
		self.run_or_pass_cols = config['run_or_pass_cols']
		self.choose_rusher = joblib.load('models/choose_rusher.joblib')
		self.choose_rusher_cols = config['choose_rusher_cols']
		self.play_encoding = config['play_encoding']
		self.wind = random.randint(0,10)
		self.temp = random.randint(40,90)
		self.play_functions = {
			'field_goal': self.field_goal,
			'no_play': self.run_play,
			'pass': self.pass_play,
			'punt': self.punt,
			'qb_kneel': self.qb_kneel,
			'qb_spike': self.qb_spike,
			'run': self.run_play
			}
		
		
	def switch_poss(self):
		self.possession = self.away if self.possession == self.home else self.home 
		self.ball_position = 100 - min(self.ball_position, 100)
		self.down = 1
		self.togo = min(10, (100-self.ball_position))
		return
	
	def kickoff(self):
		self.switch_poss()
		self.ball_position = 35
		pass
	
	def start_game(self):
		lost_kickoff = random.choice((self.home, self.away))
		self.possession=lost_kickoff
		self.kickoff()
		print(f'{self.possession.name} has won the kickoff')
		self.log_play()
		self.log_play_type('kickoff',0)

	def play(self, team):
		self.log_play()
		team.plays += 1
		features = [self.pbp[-1][key] for key in self.run_or_pass_cols]
		preds = self.run_or_pass.predict_proba([features])
		play_type_int = np.random.choice(len(preds[0]), p=preds[0])
		play_type = self.play_encoding.get(play_type_int, 1)
		yds = self.play_functions[play_type](team)
		self.togo -= yds
		self.ball_position += yds
		self.clock -= random.randint(10, 40)
		self.td_check(team)
		self.check_downs(team)
		self.log_play_type(play_type, yds)


	def log_play(self):
		"""Logs the context of the game state at each play."""
		play_data = {
			"possession": self.possession.name,
			"quarter": self.quarter,
			"down": self.down,
			"ydstogo": self.togo,
			"goal_to_go": int(self.ball_position >= 100-self.togo),
			"yardline_100": self.ball_position,
			"total_home_score": self.home.score,
			"total_away_score": self.away.score,
			"posteam_score": self.possession.score,
			"score_differential": (self.home.score - self.away.score),
			"wind": self.wind,
			"temp": self.temp,
			"quarter_seconds_remaining": self.clock,
			'half_seconds_remaining': self.clock + (900 * (self.quarter%2)),
    		'game_seconds_remaining': self.clock + (900 * (4-self.quarter)),
			'spread_line':-2.5,
			'total_line': 42.5
		}

		self.pbp.append(play_data)

	def log_play_type(self,play_type, yds,verbose=True):
		self.pbp[-1]['play_type'] = play_type
		self.pbp[-1]['yards_gained'] = yds
		if verbose:
			print(
				f'{self.possession.name} {play_type} for {yds} yards, {self.pbp[-1]['yardline_100']} yd line,' + 
				f' {self.pbp[-1]['ydstogo']} yds to go on {self.pbp[-1]['down']} down.' + 
				f' {self.pbp[-1]['quarter_seconds_remaining'] // 60}:{self.pbp[-1]['quarter_seconds_remaining']  % 60} left'
			)
		return

	def run_play(self, team):
		raw_features = dict(self.pbp[-1])
		raw_features.update(team.rb_stats)
		features = [raw_features[key] for key in self.choose_rusher_cols]
		preds = self.choose_rusher.predict_proba([features])
		rusher_depth = np.random.choice(len(preds[0]), p=preds[0])
		for x in team.RBs:
			if x.depth_team == rusher_depth:
				player = x
				break
		player.carries += 1
		yds = random.randint(-5, 15)
		player.rush_yards += yds
		return yds

	def pass_play(self, team):
		passer = team.QBs[0]
		receiver = random.choice(team.WRs)
		passer.attempts += 1
		receiver.targets += 1
		if random.randint(0, 100) > 50:
			passer.completions += 1
			receiver.receptions += 1
			yds = random.randint(-1, 30)
			receiver.receiving_yards += yds
			passer.pass_yards += yds
		else:
			yds = 0
		return yds
	
	
	def punt(self, team):
		self.switch_poss()
		self.ball_position -= random.randint(30, 60)
		if self.ball_position < 0:
			self.ball_position = 25
		return 0
	
	def field_goal(self,team):
		result = random.randint(0,100)
		if result > 50:
			team.score += 3
			self.switch_poss()
			self.ball_position = 35
			print(f'{team.name} scored a FG')
		else:
			print(f'{team.name} missed FG')
			self.switch_poss()
		return 0
	
	def qb_kneel(self, team):
		# Implementation of qb kneel play
		print("QB kneel executed.")
		return -1

	def qb_spike(self, team):
		# Implementation of qb spike play
		print("QB spike executed.")
		return 0
	
	def td_check(self, team):
		if self.ball_position > 100:
			team.score += 7
			self.kickoff()
			print(f'{team.name} scored a TD')
		return
	
	def check_downs(self, team):
		if self.togo <= 0:
			self.togo = 10
			self.down = 1
		elif self.down == 4:
			self.switch_poss()
		else:
			self.down += 1


	def play_quarter(self):
		self.clock = 900
		while self.clock > 0:
			self.play(self.possession)
		self.quarter += 1
		print(f'{self.home.name}:{self.home.score}')
		print(f'{self.away.name}:{self.away.score}')

	def play_game(self):
		while self.quarter <= 4:
			self.play_quarter()
			print(self.quarter)


In [911]:
vik.QBs

[QB:Jalen Hurts has 0 completions for 0 yards,
 QB:Kenny Pickett has 0 completions for 0 yards,
 QB:Tanner McKee has 0 completions for 0 yards]

In [912]:
with open('models/feature_config.yaml', 'r') as file:
    config = yaml.safe_load(file)
game = GameState(vik, det, config)
game.start_game()
game.play_game()


DET has won the kickoff
DET kickoff for 0 yards, 35 yd line, 10 yds to go on 1 down. 15:0 left
DET run for 4 yards, 35 yd line, 10 yds to go on 1 down. 15:0 left
DET run for 3 yards, 39 yd line, 6 yds to go on 2 down. 14:38 left
DET pass for 27 yards, 42 yd line, 3 yds to go on 3 down. 14:16 left
DET run for 9 yards, 69 yd line, 10 yds to go on 1 down. 13:59 left
DET run for 9 yards, 78 yd line, 1 yds to go on 2 down. 13:37 left
DET pass for 0 yards, 87 yd line, 10 yds to go on 1 down. 13:26 left
DET run for 8 yards, 87 yd line, 10 yds to go on 2 down. 12:50 left
DET scored a TD
PHI pass for 24 yards, 95 yd line, 2 yds to go on 3 down. 12:18 left
PHI pass for 0 yards, 35 yd line, 10 yds to go on 2 down. 11:51 left
PHI pass for 0 yards, 35 yd line, 10 yds to go on 3 down. 11:21 left
PHI missed FG
DET field_goal for 0 yards, 35 yd line, 10 yds to go on 4 down. 11:6 left
DET pass for 0 yards, 65 yd line, 10 yds to go on 2 down. 10:48 left
DET pass for 0 yards, 65 yd line, 10 yds to go on 

In [913]:
print(game.away.plays, game.home.plays)

76 69


In [914]:
pbp = pd.json_normalize(game.pbp)
pbp

,possession,quarter,down,ydstogo,goal_to_go,yardline_100,total_home_score,total_away_score,posteam_score,score_differential,wind,temp,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,spread_line,total_line,play_type,yards_gained
0,DET,1,1,10,0,35,0,0,0,0,6,50,900,1800,3600,-2.5,42.5,kickoff,0
1,DET,1,1,10,0,35,0,0,0,0,6,50,900,1800,3600,-2.5,42.5,run,4
2,DET,1,2,6,0,39,0,0,0,0,6,50,878,1778,3578,-2.5,42.5,run,3
3,DET,1,3,3,0,42,0,0,0,0,6,50,856,1756,3556,-2.5,42.5,pass,27
4,DET,1,1,10,0,69,0,0,0,0,6,50,839,1739,3539,-2.5,42.5,run,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,PHI,4,2,5,0,64,15,7,15,8,6,50,133,133,133,-2.5,42.5,run,2
142,PHI,4,3,3,0,66,15,7,15,8,6,50,100,100,100,-2.5,42.5,run,6
143,PHI,4,1,10,0,72,15,7,15,8,6,50,69,69,69,-2.5,42.5,run,7
144,PHI,4,2,3,0,79,15,7,15,8,6,50,45,45,45,-2.5,42.5,run,6


In [915]:
features = [game.pbp[-5][key] for key in game.run_or_pass_cols]
preds = game.run_or_pass.predict_proba([features])
features

[64, 2, 0, 5, 15, 8, 133, 133, 133, 6, 50, -2.5, 42.5]

In [916]:
vik.RBs

[RB:Saquon Barkley has 25 carries for 117 yards,
 RB:Kenneth Gainwell has 5 carries for 32 yards,
 RB:Will Shipley has 0 carries for 0 yards]

In [917]:
det.RBs

[RB:David Montgomery has 13 carries for 49 yards,
 RB:Jahmyr Gibbs has 17 carries for 76 yards,
 RB:Craig Reynolds has 0 carries for 0 yards,
 RB:Sione Vaki has 0 carries for 0 yards]

In [918]:
pbp.head(15)

,possession,quarter,down,ydstogo,goal_to_go,yardline_100,total_home_score,total_away_score,posteam_score,score_differential,wind,temp,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,spread_line,total_line,play_type,yards_gained
0,DET,1,1,10,0,35,0,0,0,0,6,50,900,1800,3600,-2.5,42.5,kickoff,0
1,DET,1,1,10,0,35,0,0,0,0,6,50,900,1800,3600,-2.5,42.5,run,4
2,DET,1,2,6,0,39,0,0,0,0,6,50,878,1778,3578,-2.5,42.5,run,3
3,DET,1,3,3,0,42,0,0,0,0,6,50,856,1756,3556,-2.5,42.5,pass,27
4,DET,1,1,10,0,69,0,0,0,0,6,50,839,1739,3539,-2.5,42.5,run,9
5,DET,1,2,1,0,78,0,0,0,0,6,50,817,1717,3517,-2.5,42.5,run,9
6,DET,1,1,10,0,87,0,0,0,0,6,50,806,1706,3506,-2.5,42.5,pass,0
7,DET,1,2,10,0,87,0,0,0,0,6,50,770,1670,3470,-2.5,42.5,run,8
8,DET,1,3,2,0,95,0,0,0,0,6,50,738,1638,3438,-2.5,42.5,pass,24
9,PHI,1,2,10,0,35,0,7,0,-7,6,50,711,1611,3411,-2.5,42.5,pass,0


In [919]:
pbp.loc[pbp.down==1]

,possession,quarter,down,ydstogo,goal_to_go,yardline_100,total_home_score,total_away_score,posteam_score,score_differential,wind,temp,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,spread_line,total_line,play_type,yards_gained
0,DET,1,1,10,0,35,0,0,0,0,6,50,900,1800,3600,-2.5,42.5,kickoff,0
1,DET,1,1,10,0,35,0,0,0,0,6,50,900,1800,3600,-2.5,42.5,run,4
4,DET,1,1,10,0,69,0,0,0,0,6,50,839,1739,3539,-2.5,42.5,run,9
6,DET,1,1,10,0,87,0,0,0,0,6,50,806,1706,3506,-2.5,42.5,pass,0
20,DET,1,1,10,0,64,3,7,7,-4,6,50,433,1333,3133,-2.5,42.5,run,-1
34,DET,1,1,10,0,46,6,7,7,-1,6,50,59,959,2759,-2.5,42.5,run,8
39,PHI,2,1,10,0,22,6,7,6,-1,6,50,828,828,2628,-2.5,42.5,pass,0
50,DET,2,1,10,0,56,9,7,7,2,6,50,582,582,2382,-2.5,42.5,run,15
51,DET,2,1,10,0,71,9,7,7,2,6,50,559,559,2359,-2.5,42.5,no_play,2
53,DET,2,1,10,0,81,9,7,7,2,6,50,505,505,2305,-2.5,42.5,pass,0
